## Data Set Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
from sklearn.preprocessing import LabelEncoder
sex_encoder = LabelEncoder()
df['Sex'] = sex_encoder.fit_transform(df['Sex'])

Male = 1 and Female = 0

In [5]:
correlation = df.corr()
correlation['Survived'].sort_values(ascending = False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Sex           -0.543351
Name: Survived, dtype: float64

Use just some of the basic parameters, which are age, pclass, and sex

In [6]:
parameters = ['Sex', 'Age', 'Pclass']
X = df[parameters]
y = df['Survived']

In [7]:
X = pd.concat([X, pd.get_dummies(X['Pclass'])], axis = 1)
X = X.drop('Pclass', axis = 1)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Sex    891 non-null int64
Age    714 non-null float64
1      891 non-null uint8
2      891 non-null uint8
3      891 non-null uint8
dtypes: float64(1), int64(1), uint8(3)
memory usage: 16.6 KB


In [ ]:
X

In [17]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')
imputer.fit(X)
imputer.statistics_

array([ 1., 28.,  0.,  0.,  1.])

In [18]:
X_tr = imputer.transform(X)
X = pd.DataFrame(X_tr, columns = X.columns)

In [19]:
X

,Sex,Age,1,2,3
0,1.0,22.0,0.0,0.0,1.0
1,0.0,38.0,1.0,0.0,0.0
2,0.0,26.0,0.0,0.0,1.0
3,0.0,35.0,1.0,0.0,0.0
4,1.0,35.0,0.0,0.0,1.0
5,1.0,28.0,0.0,0.0,1.0
6,1.0,54.0,1.0,0.0,0.0
7,1.0,2.0,0.0,0.0,1.0
8,0.0,27.0,0.0,0.0,1.0
9,0.0,14.0,0.0,1.0,0.0


In [21]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X['Age'] = scaler.fit_transform(X['Age'].values.reshape(len(X),1),1)

In [22]:
X

,Sex,Age,1,2,3
0,1.0,0.271174,0.0,0.0,1.0
1,0.0,0.472229,1.0,0.0,0.0
2,0.0,0.321438,0.0,0.0,1.0
3,0.0,0.434531,1.0,0.0,0.0
4,1.0,0.434531,0.0,0.0,1.0
5,1.0,0.346569,0.0,0.0,1.0
6,1.0,0.673285,1.0,0.0,0.0
7,1.0,0.019854,0.0,0.0,1.0
8,0.0,0.334004,0.0,0.0,1.0
9,0.0,0.170646,0.0,1.0,0.0


In [23]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [25]:
model = Sequential()

model.add(Dense(input_dim = 5, units=32, activation='relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(units=16, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.preprocessing import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2)
model.fit(X_train, y_train, batch_size = 10, epochs = 500)